In [3]:
import os
os.environ["PYTHONWARNINGS"] = "ignore"

In [6]:
import random
import time
import numpy as np
import math 
import cv2
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import PPO 
from typing import Callable
import os
import carla

In [7]:
SECONDS_PER_EPISODE = 25

N_CHANNELS = 3
HEIGHT = 240
WIDTH = 320


SHOW_PREVIEW = True

FIXED_DELTA_SECONDS = 0.2


In [8]:
class CarEnv(gym.Env):
    SHOW_CAM = SHOW_PREVIEW
    STEER_AMT = 1.0
    im_width = WIDTH
    im_height = HEIGHT
    front_camera = None
    CAMERA_POS_Z = 1.3 
    CAMERA_POS_X = 1.4
    discrete_actions = { 0: [0, 0.5]}
    
    def __init__(self):
        super(CarEnv, self).__init__()
        
        self.actor_list = []

        self.action_space = spaces.Discrete(len(self.discrete_actions))

        self.observation_space = spaces.Box(low=0.0, high=255.0,
                                            shape=(HEIGHT, WIDTH, N_CHANNELS), dtype=np.uint8)

        self.client = carla.Client("localhost", 2000)
        self.client.set_timeout(4.0)

        self.client.load_world('Town01')

        self.world = self.client.get_world()


        self.settings = self.world.get_settings()
        self.settings.no_rendering_mode = False
        self.settings.synchronous_mode = True  
        self.settings.fixed_delta_seconds = FIXED_DELTA_SECONDS

        self.world.apply_settings(self.settings)

        self.blueprint_library = self.world.get_blueprint_library()
        self.model_3 = self.blueprint_library.filter("model3")[0]

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.cleanup()
        
        self.collision_hist = []
        self.actor_list = []
        self.spawn_points = self.world.get_map().get_spawn_points()
        
        self.transform = self.spawn_points[2]
        
        # Define point A and point B
        self.point_a = self.transform.location
        self.point_b = self.spawn_points[241].location
        
        print("Trying to spawn vehicle...")
        self.vehicle = self.world.try_spawn_actor(self.model_3, self.transform)
        
        if self.vehicle is None:
            print("Failed to spawn vehicle! Resetting environment...")
            self.cleanup()  # Clean up any existing actors
            return self.reset()  # Retry resetting the environment
        
        print("Vehicle spawned successfully!")
        self.actor_list.append(self.vehicle)
        
        self.sem_cam = self.blueprint_library.find('sensor.camera.semantic_segmentation')
        self.sem_cam.set_attribute("image_size_x", f"{self.im_width}")
        self.sem_cam.set_attribute("image_size_y", f"{self.im_height}")
        self.sem_cam.set_attribute("fov", f"90")


        camera_init_trans = carla.Transform(carla.Location(z=self.CAMERA_POS_Z,x=self.CAMERA_POS_X))
        self.sensor = self.world.spawn_actor(self.sem_cam, camera_init_trans, attach_to=self.vehicle)
        self.actor_list.append(self.sensor)
        self.sensor.listen(lambda data: self.process_img(data))

        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
        time.sleep(2) #not detect a collision when the car spawns/falls from sky.

        colsensor = self.blueprint_library.find("sensor.other.collision")
        self.colsensor = self.world.spawn_actor(colsensor, camera_init_trans, attach_to=self.vehicle)
        self.actor_list.append(self.colsensor)
        self.colsensor.listen(lambda event: self.collision_data(event))
     

        while self.front_camera is None:
            time.sleep(0.01)#Just in case car takes any longer, because we need to be certain the car is done falling from the sky on spawn.

        self.episode_start = time.time()
        
        self.step_counter = 0
        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
        
        #self.world.tick()
        
        # showing camera at the spawn point
        if self.SHOW_CAM:
            cv2.namedWindow('Sem Camera',cv2.WINDOW_AUTOSIZE)
            cv2.imshow('Sem Camera', self.front_camera)
            cv2.waitKey(1)
        
        observation = self.front_camera

        return observation.astype(np.uint8), {}
        
    def step(self, action):
        self.step_counter +=1
        steer, throttle = self.discrete_actions[action]
        
        v = self.vehicle.get_velocity()
        kmh = int(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2))
        
        
        self.vehicle.apply_control(carla.VehicleControl(throttle= throttle, steer=steer, brake = 0.0))
        
        #self.world.tick()

        
        distance_to_destination = self.vehicle.get_location().distance(self.point_b)
        distance = self.point_a.distance(self.point_b)
        
        # storing camera to return at the end in case the clean-up function destroys it
        cam = self.front_camera
        # showing image
        if self.SHOW_CAM:
            cv2.imshow('Sem Camera', cam)
            cv2.waitKey(1)

        # start defining reward from each step
        done = False
        reward = 0
        #punish for collision
        if len(self.collision_hist) != 0:
            print("Collision")
            reward = reward - 5
            done = True
            observation = self.reset()[0]
        
        # Calculate reward based on distance traveled towards point B   
        if self.vehicle.get_location() != self.point_b:
            reward = reward + 1 / distance_to_destination  # Inverse proportional to distance
            done = False
        else:
            reward = reward + 5
            print("GOAL")
            done = True
            observation = self.reset()[0]
            
        '''
        if distance_to_destination < distance/4:  # threshold for reaching destination
            reward = 1/distance_to_destination 
        elif distance_to_destination < distance/3:
            reward = 1/distance_to_destination  
        elif distance_to_destination < distance/2:
            reward = 1/distance_to_destination  
        elif distance_to_destination < distance:
            reward = 1/distance_to_destination  
        
        else:
            reward = -1  # Negative reward for not reaching destination
            done = False
         
        #reward for acceleration
        if kmh < 15:
            reward = -0.9
        elif kmh <25:
            reward = -0.5
        elif kmh <50:
            reward = -0.1
        elif kmh>120:
            reward = -1 #punish for going too fast
        else:
            reward = 1
        '''
        # check for episode duration
        if self.episode_start + SECONDS_PER_EPISODE < time.time():
            done = True
            self.cleanup()
            #print(self.actor_list)
            
        observation = cam

        return observation.astype(np.uint8), reward, done, False, {}
        
    def cleanup(self):
        for actor in self.actor_list:
            if actor.is_alive:
                actor.destroy()
        cv2.destroyAllWindows()

        
    def process_img(self, image):
        image.convert(carla.ColorConverter.CityScapesPalette)
        i = np.array(image.raw_data)
        i = i.reshape((self.im_height, self.im_width, 4))[:, :, :3] # this is to ignore the 4th Alpha channel - up to 3
        self.front_camera = i/255.0

    def collision_data(self, event):
        self.collision_hist.append(event)
        

In [4]:
env = CarEnv()
obs, info = env.reset()
n_steps = 1000000
for _ in range(n_steps):
    # Random action
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        obs, info = env.reset()

Trying to spawn vehicle...
Vehicle spawned successfully!


KeyboardInterrupt: 

In [ ]:
env = CarEnv()

check_env(env)

In [9]:
models_dir = f"models/{int(time.time())}/"
logdir = f"logs/{int(time.time())}/"

if not os.path.exists(models_dir):
    os.makedirs(models_dir)

if not os.path.exists(logdir):
    os.makedirs(logdir)


env = CarEnv()

env.reset()

model = PPO('CnnPolicy', env, verbose=1,learning_rate=0.001, n_steps =1024, tensorboard_log=logdir)

TIMESTEPS = 500_000 # individual steps
iters = 0
while iters<4:  # how many training iterations you want
    iters += 1
    print('Iteration ', iters,' is to start...')
    try:
        model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name=f"PPO")
        print('Iteration', iters, 'has been trained')
        model.save(f"{models_dir}/{TIMESTEPS*iters}")
    except ValueError as e:
        print(f"Error during training iteration {iters}: {e}")
        continue

Trying to spawn vehicle...
Vehicle spawned successfully!


KeyboardInterrupt: 

In [5]:
'''
discrete_actions = {
    0: [-1, 1], 1: [0, 1], 2: [1, 1], 3: [0, 0]}
Action 0: Steer left (-1) and throttle (1)
Action 1: Do not steer (0) and throttle (1)
Action 2: Steer right (1) and throttle (1)
Action 3: Do not steer (0) and do not throttle (0)
'''

'\ndiscrete_actions = {\n    0: [-1, 1], 1: [0, 1], 2: [1, 1], 3: [0, 0]}\nAction 0: Steer left (-1) and throttle (1)\nAction 1: Do not steer (0) and throttle (1)\nAction 2: Steer right (1) and throttle (1)\nAction 3: Do not steer (0) and do not throttle (0)\n'

In [6]:
#discrete_actions = {
 #   0: [-1, 1], 1: [0, 1], 2: [1, 1], 3: [0, 0]}

In [7]:
#len(discrete_actions)